In [1]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image


/home/setup/miniconda3/envs/vision/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


model_name="Qwen/Qwen2-VL-2B-Instruct"

# default: Load the model on the available device(s)
model = Qwen2VLForConditionalGeneration.from_pretrained(
   model_name , torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained(
  model_name 
)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.41it/s]


In [3]:

#推論関数
def process_image(
    model,processor,
    img_path,
query="中身を解析して",
max_new_tokens=128,
):

    # 画像の準備
    image = Image.open(img_path)

    # メッセージの準備
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                },
                {"type": "text", "text": query},
            ],
        }
    ]

    # 入力の準備
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(
        text=[text_prompt], 
        images=[image], 
        padding=True, 
        return_tensors="pt"
    )
    inputs = inputs.to("cuda")

    # 推論の実行
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    return output_text

#推論
path="../250107vision_embedding_test/images/random_frame_0.jpg"
query="この画像は何?"
text=process_image(model,processor,path,query)

In [4]:
text

['この画像は、科学実験や研究の場所を示しています。テーブルの上には、電子レンジや電子秤、試験管などの機器が配置されています。背景には、パソコンやモニター、キーボードなどが見られます。この場所は、科学実験や研究のための設備が整っていると推測できます。']

In [7]:
# Messages containing multiple images and a text query
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": f"{path}"},
            {"type": "image", "image": f"{path}"},
            {"type": "text", "text": "２つの画像の共通点と相違点を教えて"},
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference
generated_ids = model.generate(**inputs, max_new_tokens=1280)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


['この画像は、科学実験や研究の様子を示しています。以下に共通点と相違点を挙げます。\n\n共通点:\n1. テーブルに設置されている機器や装置。\n2. テーブルの上に置かれた小さな容器や試料。\n3. テーブルの上に設置された電子秤。\n4. テーブルの上に設置されたマウスやキーボード。\n5. テーブルの上に設置されたコンピュータやモニター。\n\n相違点:\n1. テーブルの形状や寸法が異なる。\n2. テーブルの上に設置された機器や装置が異なる。\n3. テーブルの上に置かれた試料や容器が異なる。\n4. テーブルの上に設置された電子秤が異なる。\n5. テーブルの上に設置されたマウスやキーボードが異なる。\n6. テーブルの上に設置されたコンピュータやモニターが異なる。\n\nこれらの違いは、実験の種類や目的、研究の場所などにより異なります。']
